In [1]:
import pandas as pd
from collections import defaultdict

# All Required Inputs Are Here
# Remember to change

input_file = "Apriori_partial.xlsx"
input_sheet = "Apriori"
target_item = "Milk"

In [2]:
def read_excel_data(file_path, input_sheet=0):
    df = pd.read_excel(file_path, header=None, sheet_name=input_sheet)
    original_data = []
    for _, row in df.iterrows():
        items = [str(cell).strip() for cell in row if pd.notna(cell) and str(cell).strip() != '']
        if items == []:
            break
        original_data.append(items)
    return original_data

original_data = read_excel_data(input_file, input_sheet)

for i in original_data:
    print(i)

['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt']
['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt']
['Milk', 'Apple', 'Kidney Beans', 'Eggs']
['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt']
['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']


In [3]:
def process_cell(cell):
    cell = str(cell).strip()
    return [cell] if cell != '' else []

# get unique items
all_items = set()
for row in original_data:
    for cell in row:
        items = process_cell(cell)
        all_items.update(items)
all_items = sorted(all_items)  # Sort by alphabet

# count on every transaction
result = []
for txn_id, row in enumerate(original_data, start=1):
    counts = defaultdict(int)
    for cell in row:
        for item in process_cell(cell):
            counts[item] = 1
            break
    # build row data
    row_data = {'Txn': txn_id}
    for item in all_items:
        row_data[item] = counts.get(item, 0)
    result.append(row_data)


#  create DataFrame and set Txn as index
df = pd.DataFrame(result).set_index('Txn')
df = df[sorted(df.columns)]  # Sort by alphabet

print(df)

     Apple  Corn  Dill  Eggs  Ice cream  Kidney Beans  Milk  Nutmeg  Onion  \
Txn                                                                          
1        0     0     0     1          0             1     1       1      1   
2        0     0     1     1          0             1     0       1      1   
3        1     0     0     1          0             1     1       0      0   
4        0     1     0     0          0             1     1       0      0   
5        0     1     0     1          1             1     0       0      1   

     Unicorn  Yogurt  
Txn                   
1          0       1  
2          0       1  
3          0       0  
4          1       1  
5          0       0  


In [4]:
# calculate sup
total = df.sum()
sup = total / len(df)

# build result pd
item_calculation = pd.DataFrame({
    "Total": total,
    "Sup": sup.round(2)
}).T

print("Item Total and Sup:")
print(item_calculation)

Item Total and Sup:
       Apple  Corn  Dill  Eggs  Ice cream  Kidney Beans  Milk  Nutmeg  Onion  \
Total    1.0   2.0   1.0   4.0        1.0           5.0   3.0     2.0    3.0   
Sup      0.2   0.4   0.2   0.8        0.2           1.0   0.6     0.4    0.6   

       Unicorn  Yogurt  
Total      1.0     3.0  
Sup        0.2     0.6  


In [5]:
# create a set list for column name and correlated item
# for example, [('MA', 'Apple'), ('MC', 'Corn')...]

column_config = [(target_item[0].upper()+item[0], item) for item in list(df.columns) if item.strip().lower() != target_item.strip().lower()]
print(column_config)

# create correlation table
correlated_table = pd.DataFrame(index=df.index)

# check every item in column_config list
for new_col, item in column_config:
    try:
        # check if milk and correlated item exist
        milk_values = df[target_item]
        item_values = df[item]
        
        # if both two items > 0, then return 1; else return 0
        correlated_table[new_col] = ((milk_values > 0) & (item_values > 0)).astype(int)
    except KeyError:
        # if item not exists, return 0
        correlated_table[new_col] = 0

print("\n Correlated Table Result:")
print(correlated_table)

[('MA', 'Apple'), ('MC', 'Corn'), ('MD', 'Dill'), ('ME', 'Eggs'), ('MI', 'Ice cream'), ('MK', 'Kidney Beans'), ('MN', 'Nutmeg'), ('MO', 'Onion'), ('MU', 'Unicorn'), ('MY', 'Yogurt')]

 Correlated Table Result:
     MA  MC  MD  ME  MI  MK  MN  MO  MU  MY
Txn                                        
1     0   0   0   1   0   1   1   1   0   1
2     0   0   0   0   0   0   0   0   0   0
3     1   0   0   1   0   1   0   0   0   0
4     0   1   0   0   0   1   0   0   1   1
5     0   0   0   0   0   0   0   0   0   0


In [6]:
target_total = item_calculation[target_item]["Total"]
target_sup = item_calculation[target_item]["Sup"]
total = correlated_table.sum()
sup = total / len(correlated_table)
confi = (sup / target_sup).fillna(0)

lift = []
leverage = []
for col, item in column_config:
    xy_sup = sup[col]
    y_sup = item_calculation[item]["Sup"]

    # calculate Lift
    lift_value = xy_sup / (target_sup * y_sup) if target_sup * y_sup != 0 else 0
    
    # calculate Leverage
    leverage_value = xy_sup - (target_sup * y_sup)
    
    lift.append(lift_value)
    leverage.append(leverage_value)

lift = pd.Series(lift, index=total.index)
leverage = pd.Series(leverage, index=total.index)

calculation_corrected_table = pd.DataFrame({
    "Total": total,
    "Sup": sup.round(2),
    "confi": confi.apply(lambda x: f"{int(round(x*100))}%"),
    "Lift": lift.round(5),
    "Leverage": leverage.round(5)
}).T

print("Correlation Calculation")
print(calculation_corrected_table)

Correlation Calculation
               MA       MC    MD       ME    MI    MK       MN       MO  \
Total           1        1     0        2     0     3        1        1   
Sup           0.2      0.2   0.0      0.4   0.0   0.6      0.2      0.2   
confi         33%      33%    0%      67%    0%  100%      33%      33%   
Lift      1.66667  0.83333   0.0  0.83333   0.0   1.0  0.83333  0.55556   
Leverage     0.08    -0.04 -0.12    -0.08 -0.12   0.0    -0.04    -0.16   

               MU       MY  
Total           1        2  
Sup           0.2      0.4  
confi         33%      67%  
Lift      1.66667  1.11111  
Leverage     0.08     0.04  
